In [1]:
from pytorch_lightning import LightningModule, LightningDataModule
from torchmetrics.classification import MultilabelAUROC
from torchmetrics import MetricCollection, MeanMetric
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import random
import numpy as np
import pandas as pd
import warnings
import os
from PIL import Image

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(
    action='ignore', category=pd.errors.SettingWithCopyWarning)


class ISICDatast(Dataset):
    def __init__(self,
                 df,
                 img_dir,
                 transform=None,
                 **kwargs):
        df = df.reset_index(drop=True)
        self.transform = transform
        self.df = df
        self.img_dir = img_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.loc[idx, 'isic_id']
        img_path = os.path.join(self.img_dir, img_id + '.jpg')
        img = Image.open(img_path).convert('RGB')
        label = self.df.loc[idx, 'target'] # 0 or 1
        label = torch.tensor([label, 1-label], dtype=torch.float32)
        if self.transform:
            img = self.transform(image=np.array(img))['image']
        return img, label


class ISICDataModule(LightningDataModule):
    def __init__(self, train_dataset, val_dataset, batch_size=64):
        super().__init__()
        self.batch_size = batch_size
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset

    def train_dataloader(self):
        return DataLoader(dataset=self.train_dataset,
                          batch_size=self.batch_size,
                          num_workers=4,
                          pin_memory=False,
                          shuffle=True,
                          )

    def val_dataloader(self):
        return DataLoader(dataset=self.val_dataset,
                          batch_size=self.batch_size,
                          num_workers=4,
                          pin_memory=False,
                          shuffle=False,
                          )


class CutMix:
    def __init__(self,
                 mode: str = 'horizontal',
                 p: float = 1.0,
                 cuts_num: int = 1):
        assert mode in ['horizontal']
        self.mode = mode
        self.cuts_num = cuts_num
        self.p = p

    def apply_horizontal(self, imgs, labels):
        w = imgs.shape[-1]
        b = imgs.shape[0]

        alphas = np.sort(np.random.rand(self.cuts_num))
        rand_index = [np.random.permutation(b) for _ in range(self.cuts_num)]
        imgs_tomix = [imgs[idxes] for idxes in rand_index]
        labels_tomix = [labels[idxes] for idxes in rand_index]

        for alpha, img_tomix in zip(alphas, imgs_tomix):
            imgs[..., int(alpha*w):] = img_tomix[..., int(alpha*w):]

        labels = labels*alphas[0]
        for i in range(1, self.cuts_num):
            labels += labels_tomix[i-1]*(alphas[i] - alphas[i-1])
        labels += labels_tomix[-1] * (1 - alphas[-1])

        return imgs, labels

    def __call__(self, imgs, labels):
        if random.random() > self.p:
            return imgs, labels
        if self.mode in ['horizontal']:
            imgs, labels = self.apply_horizontal(imgs, labels)
        return imgs, labels


class LitCls(LightningModule):

    def __init__(
            self,
            model: torch.nn.Module,
            learning_rate: float = 3e-4,
            cutmix_p: float = 0,
            cuts_num: int = 1,
    ) -> None:
        super().__init__()

        self.model: torch.nn.Module = model
        self.learning_rate: float = learning_rate
        self.aug_cutmix = CutMix(
            mode='horizontal', p=cutmix_p, cuts_num=cuts_num)

        self.loss: torch.nn.Module = nn.CrossEntropyLoss()
        metric_ce = MetricCollection({
            "CE": MeanMetric()
        })
        metric_auroc = MetricCollection({
            "AUROC": MultilabelAUROC(num_labels=2, average="macro"),
        })

        self.train_ce: MetricCollection = metric_ce.clone(prefix="train_")
        self.val_ce: MetricCollection = metric_ce.clone(prefix="val_")
        self.train_auroc: MetricCollection = metric_auroc.clone(
            prefix="train_")
        self.val_auroc: MetricCollection = metric_auroc.clone(prefix="val_")

    def training_step(self, batch, batch_idx: int) -> torch.Tensor:
        x, y = batch
        x = x.transpose(1, 3) # (B, H, W, C) -> (B, C, H, W)
        x, y = self.aug_cutmix(x, y)
        preds = self.model(x)
        train_loss = self.loss(preds, y)
        self.train_ce(train_loss)
        self.log('train_loss', train_loss, prog_bar=True, sync_dist=True)
        self.train_auroc(F.sigmoid(preds), (y+0.9).int())
        return train_loss

    def on_train_epoch_end(self) -> None:
        self.log_dict(self.train_ce.compute(), sync_dist=True)
        self.train_ce.reset()
        self.log_dict(self.train_auroc.compute(), sync_dist=True)
        self.train_auroc.reset()

    def validation_step(self, batch: dict[str, torch.Tensor], batch_idx: int) -> None:
        x, y = batch
        x = x.transpose(1, 3) # (B, H, W, C) -> (B, C, H, W)
        preds = self.model(x)
        val_loss = self.loss(preds, y)
        self.val_ce(val_loss)
        self.val_auroc(F.sigmoid(preds), (y+0.9).int())

    def on_validation_epoch_end(self) -> None:
        self.log_dict(self.val_ce.compute(), prog_bar=True, sync_dist=True)
        self.val_ce.reset()
        self.log_dict(self.val_auroc.compute(), prog_bar=True, sync_dist=True)
        self.val_auroc.reset()

    def configure_optimizers(self):
        optimizer = AdamW(
            params=self.trainer.model.parameters(), lr=self.learning_rate)
        scheduler = CosineAnnealingLR(optimizer, T_max=self.trainer.max_epochs)
        return [optimizer], [scheduler]


In [2]:
CFG = {
    "name": "resnet18",
    "model_name": "resnet18",
    "img_size": 224,
    "learning_rate": 3e-3,
    "epochs_num": 1,
    "batch_size": 32
}
CFG

{'name': 'resnet18',
 'model_name': 'resnet18',
 'img_size': 224,
 'learning_rate': 0.003,
 'epochs_num': 1,
 'batch_size': 32}

In [3]:
# from training_utils import ISICDatast, ISICDataModule, LitCls
import albumentations as A
import timm
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, RichProgressBar
from pytorch_lightning import Trainer
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import warnings
import argparse
from pathlib import Path

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(
    action='ignore', category=pd.errors.SettingWithCopyWarning)

models_path = 'model_weights/'


# from pytorch_lightning.loggers import WandbLogger

# import wandb


if __name__ == '__main__':

#     parser = argparse.ArgumentParser()
#     parser.add_argument(
#         '-с', '--cfg', help='Model config file path', dest='cfg_path')
#     args = {}
#     for name, value in vars(parser.parse_args()).items():
#         args[name] = value
    
    root_path = Path("/kaggle/input/isic-2024-challenge")

    train_csv_path = root_path/'train-metadata.csv'

    # load config
#     with open(args['cfg_path'], 'r') as f:
#         CFG = json.load(f)

    # read train metadata
    data = pd.read_csv(train_csv_path)
    data = data.sample(n=100)

    train_df, valid_df = train_test_split(
        data, test_size=0.1, random_state=42, stratify=data['target'])

    # create train and validation datasets
    transform = A.Compose([
        A.Resize(CFG['img_size'], CFG['img_size']),
        A.Normalize(),
    ])
    train_dataset = ISICDatast(df=train_df,
                               img_dir=root_path/'train-image/image/',
                               transform=transform)
    valid_dataset = ISICDatast(df=valid_df,
                               img_dir=root_path/'train-image/image/',
                               transform=transform)
    datamodule = ISICDataModule(train_dataset, train_dataset,
                                batch_size=CFG['batch_size'])

    # create pretrained model
    model = timm.create_model(
        CFG['model_name'], pretrained=True,
        num_classes=2
    )
    lit_cls = LitCls(model, cutmix_p=0.9, learning_rate=CFG['learning_rate'])

    # create callbacks
    checkpoint_callback = ModelCheckpoint(
        monitor=None,  # save only last
        filename='{epoch}-{val_AUROC:.3f}',
        save_last=True,
    )
    lr_monitor = LearningRateMonitor(logging_interval='epoch')
    rich_progress = RichProgressBar()

    # with open('/wandb_key.txt') as f:
    #     WANDB_KEY = f.readline()
    # wandb.login(key=WANDB_KEY)
    # logger = WandbLogger(
    #     project='BirdCLEF',
    #     log_model=True,
    # )

    # create trainer and start training process
    trainer = Trainer(
        check_val_every_n_epoch=1,
        num_sanity_val_steps=0,
        max_epochs=CFG['epochs_num'],
        accumulate_grad_batches=1,
        callbacks=[rich_progress, lr_monitor, checkpoint_callback],
        # logger=logger,
        log_every_n_steps=50,
        # accelerator='gpu',
    )
    trainer.fit(lit_cls, datamodule=datamodule)
    # wandb.finish()

    name = CFG['name'].split('/')[-1].split('.')[0]
    trainer.save_checkpoint(f'{models_path}{CFG["name"]}.ckpt')


/tmp/ipykernel_18/1515938260.py:43: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(train_csv_path)


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

2024-07-10 13:41:55.878709: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 13:41:55.878827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 13:41:56.017750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


┏━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name        ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model       │ ResNet           │ 11.2 M │
│ 1 │ loss        │ CrossEntropyLoss │      0 │
│ 2 │ train_ce    │ MetricCollection │      0 │
│ 3 │ val_ce      │ MetricCollection │      0 │
│ 4 │ train_auroc │ MetricCollection │      0 │
│ 5 │ val_auroc   │ MetricCollection │      0 │
└───┴─────────────┴──────────────────┴────────┘

Trainable params: 11.2 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 11.2 M                                                                                               
Total estimated model params size (MB): 44

Output()

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in 
targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028

/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in 
targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028

In [4]:
ls model_weights

resnet18.ckpt
